In [10]:
using Pkg
# Ensure required packages (uncomment to install if needed)
Pkg.add([
    "MLJ", 
    "MLJBase", 
    "MLJModels", 
    "MLJEnsembles", 
    "MLJLinearModels", 
    "DecisionTree", 
    "MLJDecisionTreeInterface", 
    "NaiveBayes", 
    "EvoTrees", 
    "CategoricalArrays", 
    "Random",
    "LIBSVM",           
    "Plots",            
    "MLJModelInterface", 
    "CSV",              
    "DataFrames",      
    "MLJFlux", 
    "UrlDownload",      
    "XGBoost"    
])
include("Utils.jl")
using .Utils

   Resolving package versions...
   Installed PartialFunctions ─ v1.2.1
   Installed Metalhead ──────── v0.9.5
   Installed JLD2 ───────────── v0.5.15
   Installed FileIO ─────────── v1.17.1
   Installed MLJFlux ────────── v0.6.6
    Updating `~/.julia/environments/v1.11/Project.toml`
⌃ [094fc8d1] + MLJFlux v0.6.6
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [5789e2e9] + FileIO v1.17.1
⌅ [033835bb] + JLD2 v0.5.15
⌃ [094fc8d1] + MLJFlux v0.6.6
  [dbeba491] + Metalhead v0.9.5
  [570af359] + PartialFunctions v1.2.1
        Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated -m`
Precompiling project...
    941.2 ms  ✓ PartialFunctions
   2184.2 ms  ✓ FileIO
   1380.1 ms  ✓ FileIO → HTTPExt
   2473.4 ms  ✓ Plots → FileIOExt
  17031.0 ms  ✓ JLD2
    820.5 ms  ✓ JLD2 → UnPackExt
   3777.4 ms  ✓ Metalhead
   4088.4 ms  ✓ MLJFlux

In [11]:
using CSV
using DataFrames

# 1. Load data from wdbc.data file
df = CSV.read("wdbc.data", DataFrame, header=false)

# 2. Data separation (mimicking the Python script)
y = df[:, 2]
X = df[:, 3:end-1]

# 3. Metadata equivalent
println("--- Dataset Summary ---")
println("Total Rows: ", nrow(df))
println("Total Columns: ", ncol(df))
println("Target variable (y) shape: ", size(y))
println("Features matrix (X) shape: ", size(X))

println("\n--- Variable Information (First 5 Features) ---")
display(first(describe(X), 5))
println("\n--- Target Distribution ---")
display(combine(groupby(DataFrame(Diagnosis=y), :Diagnosis), nrow))

--- Dataset Summary ---
Total Rows: 569
Total Columns: 32
Target variable (y) shape: (569,)
Features matrix (X) shape: (569, 29)

--- Variable Information (First 5 Features) ---


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,Column3,14.1273,6.981,13.37,28.11,0,Float64
2,Column4,19.2896,9.71,18.84,39.28,0,Float64
3,Column5,91.969,43.79,86.24,188.5,0,Float64
4,Column6,654.889,143.5,551.1,2501.0,0,Float64
5,Column7,0.0963603,0.05263,0.09587,0.1634,0,Float64



--- Target Distribution ---


Row,Diagnosis,nrow
,String1,Int64
1,M,212
2,B,357
